In [1]:
# Test
file_name="./aoc_2112_test3.txt"
# Prod
file_name="./aoc_2112_prod.txt"

In [2]:
with open(file_name) as f:
    raw_data=[line for line in f.read().splitlines()]
    f.close()

# Check
display(raw_data[:10])

['rf-RL',
 'rf-wz',
 'wz-RL',
 'AV-mh',
 'end-wz',
 'end-dm',
 'wz-gy',
 'wz-dm',
 'cg-AV',
 'rf-AV']

In [3]:
#data=[[int(digit) for digit in line] for line in raw_data]
data=[line for line in raw_data]

In [4]:
data[:5]

['rf-RL', 'rf-wz', 'wz-RL', 'AV-mh', 'end-wz']

## Part 0

In [5]:
from typing import List, Set, Dict
from copy import deepcopy

In [6]:
def is_cave_small(id: str) -> bool:
    return id[0].islower()

In [7]:
class Cave:
    def __init__(self, id: str):
        self.id=id
        self.is_small=is_cave_small(id[0])
        self.tunnels_in=list()
        self.tunnels_out=list()
        
    def __str__(self):
        return str(self.id)

    def add_in(self, c):
        self.tunnels_in.append(c)

    def add_out(self, c):
        self.tunnels_out.append(c)

In [8]:
class Map:
    def __init__(self, connections: List[str]):
        self.caves=dict()
        for connection in connections:
            caves=connection.split('-')
            for i in range(2):
                if caves[i] not in self.caves.keys(): self.caves[caves[i]]=Cave(caves[i])
            for i in range(2):            
                if not (self.caves[caves[i]].id =='end' or self.caves[caves[1-i]].id =='start'): 
                    self.caves[caves[i]].add_out(self.caves[caves[1-i]])
                if not (self.caves[caves[i]].id =='start' or self.caves[caves[1-i]].id =='end'): 
                    self.caves[caves[i]].add_in(self.caves[caves[1-i]])

    def print(self):
        for c in self.caves.items():
            print(f"{c[0]}\tout:{[cave.id for cave in c[1].tunnels_out]}, in:{[cave.id for cave in c[1].tunnels_in]}")
                
    def all_paths_from_cave_no_small_twice(self, from_cave:str, visited:Dict[Cave,int]) -> List[List[Cave]]:
        paths_from_this_cave=list()
        visited_before=visited.copy()
        if self.caves[from_cave].is_small: 
            visited_before[self.caves[from_cave]]=visited_before.get(self.caves[from_cave], 0)+1
        for cave in self.caves[from_cave].tunnels_out:
            #print(f"Checking from {from_cave} to {cave.id} with already visited { visited_before }...")
            if visited_before.get(cave, 0)<1:
                if cave.id=='end':
                    paths_from_next=[[cave]]
                else:
                    paths_from_next=self.all_paths_from_cave_no_small_twice(cave.id, visited_before)
                    #print(f"For {from_cave} paths from next {cave.id} are {[p.id for path in paths_from_next for p in path]}")
                for path in paths_from_next:
                    path.insert(0, self.caves[from_cave])
                    paths_from_this_cave.append(path)
        return paths_from_this_cave

    def all_paths_from_cave_one_small_twice(self, from_cave:str, visited:Dict[Cave,int]) -> List[List[Cave]]:
        paths_from_this_cave=list()
        visited_before=visited.copy()
        if self.caves[from_cave].is_small: 
            visited_before[self.caves[from_cave]]=visited_before.get(self.caves[from_cave], 0)+1
        for cave in self.caves[from_cave].tunnels_out:
            #print(f"Checking from {from_cave} to {cave.id} with already visited { visited_before }...")
            if visited_before.get(cave, 0)<2 and len(list(filter(lambda v : v==2, visited_before.values())))<2:
                if cave.id=='end':
                    paths_from_next=[[cave]]
                else:
                    paths_from_next=self.all_paths_from_cave_one_small_twice(cave.id, visited_before)
                    #print(f"For {from_cave} paths from next {cave.id} are {[p.id for path in paths_from_next for p in path]}")
                for path in paths_from_next:
                    path.insert(0, self.caves[from_cave])
                    paths_from_this_cave.append(path)
        return paths_from_this_cave

## Part 1

In [9]:
map=Map(data)
# Check
map.print()

rf	out:['RL', 'wz', 'AV', 'gy', 'dm', 'mh'], in:['RL', 'wz', 'AV', 'gy', 'dm', 'mh']
RL	out:['rf', 'wz', 'cg', 'gy'], in:['rf', 'wz', 'cg', 'gy', 'start']
wz	out:['rf', 'RL', 'end', 'gy', 'dm', 'mh'], in:['rf', 'RL', 'gy', 'dm', 'mh']
AV	out:['mh', 'cg', 'rf', 'gy'], in:['mh', 'cg', 'rf', 'gy', 'start']
mh	out:['AV', 'end', 'rf', 'qk', 'wz'], in:['AV', 'rf', 'qk', 'wz']
end	out:[], in:['wz', 'dm', 'mh']
dm	out:['end', 'wz', 'rf'], in:['wz', 'rf']
gy	out:['wz', 'rf', 'cg', 'RL', 'VI', 'AV'], in:['wz', 'rf', 'cg', 'RL', 'VI', 'AV']
cg	out:['AV', 'gy', 'RL'], in:['AV', 'gy', 'RL', 'start']
VI	out:['gy'], in:['gy']
start	out:['cg', 'RL', 'AV'], in:[]
qk	out:['mh'], in:['mh']


In [10]:
%%time
all_paths1=map.all_paths_from_cave_no_small_twice('start', dict())

CPU times: user 24.1 ms, sys: 2.22 ms, total: 26.4 ms
Wall time: 24.9 ms


In [11]:
#print([[cave.id for cave in path] for path in all_paths])
result=len(all_paths1)

In [12]:
print(f"Result for part 1: {result}")

Result for part 1: 3421


## Part 2

In [13]:
%%time
all_paths2=map.all_paths_from_cave_one_small_twice('start', dict())

CPU times: user 4.32 s, sys: 15.3 ms, total: 4.33 s
Wall time: 4.33 s


In [14]:
#display([[cave.id for cave in path] for path in all_paths])
result=len(all_paths2)

In [15]:
print(f"Result for part 2: {result}")

Result for part 2: 84870
